In [1]:
import sklearn.preprocessing as skp
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import scipy.stats as stats
import scipy.integrate as integrate
import scipy.optimize as opt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pp
from IPython.display import set_matplotlib_formats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import math
import time
from sklearn.metrics import confusion_matrix

set_matplotlib_formats('png')
REG_ROUND = 4
SPEC_ROUND = 6

### Part 1

In [2]:
white_df = pd.read_csv('winequality-white.csv', delimiter=";")
red_df = pd.read_csv('winequality-red.csv', delimiter=";")

In [3]:
white_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
red_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
white_df['color'] = 'white'
red_df['color'] = 'red'

In [6]:
df = pd.concat([white_df, red_df])

In [7]:
# randomizing
df = df.sample(frac=1)
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
156,7.1,0.43,0.42,5.5,0.070,29.0,129.0,0.99730,3.42,0.72,10.5,5,red
4033,7.5,0.26,0.38,5.7,0.021,23.0,125.0,0.99338,3.13,0.62,11.1,6,white
2089,7.9,0.17,0.32,1.6,0.053,47.0,150.0,0.99480,3.29,0.76,9.6,6,white
384,6.0,0.36,0.39,3.2,0.027,20.0,125.0,0.99100,3.38,0.39,11.3,7,white
449,6.4,0.28,0.29,1.6,0.052,34.0,127.0,0.99290,3.48,0.56,10.5,7,white


### Part 2

In [8]:
train, test = train_test_split(df, test_size=0.2)

In [9]:
# assuring balanced proportions
train_proportions = train['color'].value_counts()
test_proportions = test['color'].value_counts()

In [10]:
train_proportions /= train.shape[0]
train_proportions

white    0.756783
red      0.243217
Name: color, dtype: float64

In [11]:
test_proportions /= test.shape[0]
test_proportions

white    0.742308
red      0.257692
Name: color, dtype: float64

### Part 3

In [12]:
scaler = skp.StandardScaler()

train['color'] = train['color'].replace(('white', 'red'), (1, 0))
x_train = train[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']]

scaled = scaler.fit_transform(x_train.values)
x_train = pd.DataFrame(data=scaled, columns=x_train.columns)
# y_train
y_train = train['color']
x_train = sm.add_constant(x_train)
train['color'].value_counts()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


1    3933
0    1264
Name: color, dtype: int64

In [13]:
start = time.time()
model = sm.GLM(y_train.values, x_train, family=sm.families.Binomial()).fit()
end = time.time()
print(f'elapsed time: {end - start} seconds')

elapsed time: 0.1527106761932373 seconds


In [14]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                 5197
Model:                            GLM   Df Residuals:                     5184
Model Family:                Binomial   Df Model:                           12
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -150.18
Date:                Wed, 28 Apr 2021   Deviance:                       300.37
Time:                        02:08:42   Pearson chi2:                 6.79e+07
No. Iterations:                    11                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    5.0027      0.390     12.825      0.000       4.238       5.767
fixed acidity            0.0972      0.344      0.282      0.778      -0.578       0.772
volatile acidity        -1.0914      0.212     -5.157      0.000      -1.506      -0.677
citric acid              0.5967      0.205      2.908      0.004       0.194       0.999
residual sugar           5.4974      0.648      8.477      0.000       4.226       6.768
chlorides               -0.8949      0.186     -4.816      0.000      -1.259      -0.531
free sulfur dioxide     -1.1230      0.314     -3.576      0.000      -1.739      -0.507
total sulfur dioxide     3.0989      0.345      8.990      0.000       2.423       3.774
density                 -5.3337      0.641     -8.323      0.000      -6.590      -4.078
pH                       0.1425      0.266      0.536      0.592      -0.378       0.663
sulphates               -0.5503      0.218     -2.520      0.012      -0.978      -0.122
alcohol                 -1.9391      0.369     -5.261      0.000      -2.662      -1.217
quality                 -0.2883      0.206     -1.399      0.162      -0.692       0.116
========================================================================================
"""

The least significant values seem to be fixed acidity, pH, quality, and citric acid, as they all have P-values over 0.05.

Higher residual sugar is indicative of a white wine, since the coefficient is positive (more likely to be 1 than 0).
Higher alcohol is indicative of a red wine, since the coefficient is negative (more likely to be 0 than 1).

### Part 4

In [15]:
print(f'number of observations: {train.shape[0]}')
print(f'number of parameters: {len(model.params)}')
print(f'degrees of freedom: {model.df_model}')
print(f'log-likelihood: {round(model.llf, SPEC_ROUND)}')
print(f'Deviance: {round(model.deviance, SPEC_ROUND)}')
print(f'AIC: {round(model.aic, SPEC_ROUND)}')

number of observations: 5197
number of parameters: 13
degrees of freedom: 12
log-likelihood: -150.182536
Deviance: 300.365071
AIC: 326.365071


### Part 5

In [16]:
# confusion matrix --- positive negative
#             positive
#             positive
prediction = model.predict()
prediction = [1 if pred >= 0.5 else 0 for pred in prediction]
actual = train['color']
matrix = confusion_matrix(actual, prediction)

def print_confusion(matrix, n):  
    print('\t\t\t Actual Results')
    print('\t\t\tpositive\tnegative')
    print(f'Predicted positive\t {matrix[0][0]}\t\t{matrix[0][1]}')
    print(f'Results negative\t {matrix[1][0]}\t\t{matrix[1][1]}')
    accuracy = (matrix[0][1] + matrix[1][0]) / n
    print(f'Prediction Accuracy: {round(1 - accuracy, SPEC_ROUND)}')
    
    
print('Train Prediction Confusion Matrix')
print_confusion(matrix, train.shape[0])

test['color'] = test['color'].replace(('white', 'red'), (1, 0))
x_test = test[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']]
scaled = scaler.fit_transform(x_test.values)
x_test = pd.DataFrame(data=scaled, columns=x_test.columns)
x_test = sm.add_constant(x_test)
prediction = model.predict(x_test)
prediction = [1 if pred >= 0.5 else 0 for pred in prediction]
actual = test['color']
matrix = confusion_matrix(actual, prediction)
print('\n\nTest Prediction Confusion Matrix')
acc = print_confusion(matrix, test.shape[0])

Train Prediction Confusion Matrix
			 Actual Results
			positive	negative
Predicted positive	 1247		17
Results negative	 11		3922
Prediction Accuracy: 0.994612


Test Prediction Confusion Matrix
			 Actual Results
			positive	negative
Predicted positive	 325		10
Results negative	 2		963
Prediction Accuracy: 0.990769


/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Part 6

In [17]:
x_test, y_test = x_test, test['color']
x_train, y_train = x_train, train['color']

In [18]:
x_test.dtypes, y_test.dtypes

(const                   float64
 fixed acidity           float64
 volatile acidity        float64
 citric acid             float64
 residual sugar          float64
 chlorides               float64
 free sulfur dioxide     float64
 total sulfur dioxide    float64
 density                 float64
 pH                      float64
 sulphates               float64
 alcohol                 float64
 quality                 float64
 dtype: object,
 dtype('int64'))

In [19]:
x_train.dtypes, y_train.dtypes

(const                   float64
 fixed acidity           float64
 volatile acidity        float64
 citric acid             float64
 residual sugar          float64
 chlorides               float64
 free sulfur dioxide     float64
 total sulfur dioxide    float64
 density                 float64
 pH                      float64
 sulphates               float64
 alcohol                 float64
 quality                 float64
 dtype: object,
 dtype('int64'))

### Part 7

In [20]:
class LogisticRegression:
    
    def __init__(self, learning_rate = 0.005, epochs=20000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.elapsed_time = None
        self.error = None
        
    def fit(self, X, y):
        time_start = time.time()
        n_rows = X.shape[0]
        n_cols = X.shape[1]
        self.weights = np.ones(n_cols)
        self.error = np.inf
        
        # gradient descent
        for epoch in range(self.epochs):
            lin_model = np.dot(X, self.weights) # w * x
            y_pred = self._sigmoid(lin_model)
            
            dw = (1 / n_rows) * np.dot(X.T, y_pred - y)
            self.weights -= (self.learning_rate * dw)
            self.error = self._error(y, y_pred)
        time_end = time.time()
        self.elapsed_time = time_end - time_start
        return self.weights
    
    def _error(self, y, pred):
        return (-1 / len(y)) * (np.dot(y.transpose(), np.log(pred)) + np.dot((np.ones(len(y)) - y.transpose()), np.log(np.ones(len(y)) - pred)))
    
    def predict(self, X, sigmoided=True):
        lin_model = np.dot(X, self.weights)
        y_pred = self._sigmoid(lin_model) 
        if not sigmoided:
            return y_pred
        return [1 if y_prediction >= 0.5 else 0 for y_prediction in y_pred]
    
    def _sigmoid(self, X):
        return [1 / (1 + math.exp(-x)) for x in X]

In [21]:
x_train_arr = np.array(x_train)
y_train_arr = np.array(y_train)
log_reg = LogisticRegression()
params = log_reg.fit(x_train_arr, y_train_arr)
# params, num_epochs_required, error = gradient_solver(x_train_arr, y_train_arr, alpha_value, tolerance, num_epochs=50)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log


In [22]:
test_df = pd.DataFrame()
test_df['predicted'] = log_reg.predict(x_train_arr)
test_df['actual'] = y_train_arr

In [23]:
test_df['difference'] = [abs(result) for result in test_df['predicted'] - test_df['actual']]

In [24]:
print(f'--- Hand-made Logistic Regression metrics ---')
print(f'time elapsed: {round(log_reg.elapsed_time, REG_ROUND)} seconds')
print(f'number of epochs: {log_reg.epochs}')
print(f'final error: {round(log_reg.error, SPEC_ROUND)}')
misclass = sum(test_df['difference'])
print(f'Accuracy: {round(1 - (misclass / test_df.shape[0]), SPEC_ROUND)}')

parameter_comp_df = pd.DataFrame()
parameter_comp_df['Library Solver'] = model.params
parameter_comp_df['Handmade Logistic Regression'] = log_reg.weights
parameter_comp_df.index = x_train.columns
print('\n--- Parameter Comparison ---')
parameter_comp_df

--- Hand-made Logistic Regression metrics ---
time elapsed: 77.5005 seconds
number of epochs: 20000
final error: 0.04426
Accuracy: 0.991726

--- Parameter Comparison ---


,Library Solver,Handmade Logistic Regression
const,5.002726,2.951651
fixed acidity,0.097161,-0.809335
volatile acidity,-1.091418,-1.033560
citric acid,0.596698,0.400600
residual sugar,5.497394,0.975594
chlorides,-0.894877,-0.785803
free sulfur dioxide,-1.123019,0.162729
total sulfur dioxide,3.098883,2.014656
density,-5.333711,-1.242175
pH,0.142522,-0.614404


### Question 8

In [25]:
# train
print(f'Training Set Prediction')
matrix = confusion_matrix(test_df['predicted'], y_train_arr)
print_confusion(matrix, test_df.shape[0])

# test
print('\nTest Set Prediction')
matrix = confusion_matrix(log_reg.predict(x_test), y_test)
print_confusion(matrix, len(x_test))

Training Set Prediction
			 Actual Results
			positive	negative
Predicted positive	 1241		20
Results negative	 23		3913
Prediction Accuracy: 0.991726

Test Set Prediction
			 Actual Results
			positive	negative
Predicted positive	 324		3
Results negative	 11		962
Prediction Accuracy: 0.989231


### Question 9

These Statistics do not 100% align with the library model **because I do not have five hours to run 500,000,000 epochs**. If I did, I could run it long enough to get to the exact same parameters, but unfortunately I do not have access to the supercomputer located under Memorial Stadium

In [26]:
n = len(x_train_arr)
p = len(log_reg.weights)
y = np.array(test_df['actual'])
pred = np.array(test_df['predicted'])
log_lik = log_reg.error * -n
AIC = -2 * log_lik + 2 * (p + 1)
deviance = log_lik * -2

print(f'--- Hand-Coded Model Statistics ---')
print(f'Log-Likelihood: {round(log_lik, SPEC_ROUND)}')
print(f'AIC: {round(AIC, SPEC_ROUND)}')
print(f'Deviance: {round(deviance, SPEC_ROUND)}')

print(f'\n--- Library Model Statistics ---')
print(f'Log-Likelihood: {round(model.llf, SPEC_ROUND)}')
print(f'AIC: {round(model.aic, SPEC_ROUND)}')
print(f'Deviance: {round(model.deviance, SPEC_ROUND)}')

--- Hand-Coded Model Statistics ---
Log-Likelihood: -230.019092
AIC: 488.038184
Deviance: 460.038184

--- Library Model Statistics ---
Log-Likelihood: -150.182536
AIC: 326.365071
Deviance: 300.365071


### Question 10

In [27]:
print(f'Covariance for the Library Logistic Regression Model')
comparison_df = pd.DataFrame(data=x_train, columns=['fixed acidity', 'volatile acidity', 'citric acid', 
                                                        'residual sugar', 'chlorides', 'free sulfur dioxide', 
                                                        'total sulfur dioxide', 'density', 'pH', 'sulphates', 
                                                        'alcohol', 'quality'])
model_prediction = [1 if x >= 0.5 else 0 for x in model.predict()]
comparison_df['Library Model Prediction'] = model_prediction
comparison_df['Actual'] = test_df['actual']
model.cov_params()

Covariance for the Library Logistic Regression Model


,const,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
const,0.152156,-0.004896,-0.016347,0.002661,0.165260,-0.014869,-0.013064,0.040340,-0.058913,-0.003661,-0.017594,-0.025305,0.003468
fixed acidity,-0.004896,0.118612,0.014720,-0.010107,0.097698,0.023325,-0.007001,0.002212,-0.145742,0.063729,0.005794,-0.047165,-0.002057
volatile acidity,-0.016347,0.014720,0.044793,0.019239,0.015349,-0.004328,0.009219,-0.015926,-0.044094,0.015165,0.005538,-0.024992,0.007847
citric acid,0.002661,-0.010107,0.019239,0.042117,0.015050,-0.012455,0.000453,-0.004647,-0.023678,0.008973,0.001325,-0.015131,-0.000583
residual sugar,0.165260,0.097698,0.015349,0.015050,0.420526,0.010187,-0.045027,0.032257,-0.325629,0.085349,0.017231,-0.141153,-0.013599
chlorides,-0.014869,0.023325,-0.004328,-0.012455,0.010187,0.034521,-0.003895,0.001268,-0.021484,0.013866,0.000324,0.007114,-0.002127
free sulfur dioxide,-0.013064,-0.007001,0.009219,0.000453,-0.045027,-0.003895,0.098645,-0.068108,0.043496,-0.011721,0.001918,0.024966,-0.014926
total sulfur dioxide,0.040340,0.002212,-0.015926,-0.004647,0.032257,0.001268,-0.068108,0.118822,-0.021888,0.008180,-0.012739,0.000903,0.004419
density,-0.058913,-0.145742,-0.044094,-0.023678,-0.325629,-0.021484,0.043496,-0.021888,0.410690,-0.116163,-0.039175,0.178880,0.017611
pH,-0.003661,0.063729,0.015165,0.008973,0.085349,0.013866,-0.011721,0.008180,-0.116163,0.070640,0.002816,-0.047271,-0.005135


In [28]:
print(f'Covariance Matrix for the Hand-Coded Logistic Regression Model')
pred_no_round = np.array(log_reg.predict(x_train_arr, sigmoided=False))
Vjj = pred_no_round * (np.ones(len(x_train_arr)) - pred_no_round)
V = np.diag(Vjj)
C = np.linalg.inv(x_train_arr.transpose() @ V @ x_train_arr)
cov_df = pd.DataFrame(data=C, columns = x_train.columns)
cov_df.index = x_train.columns
cov_df

Covariance Matrix for the Hand-Coded Logistic Regression Model


,const,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
const,0.021007,-0.006500,-0.003464,0.001064,0.001149,-0.004029,0.007154,0.003731,0.011294,-0.006263,-0.006475,0.007289,0.000592
fixed acidity,-0.006500,0.041388,0.003830,-0.006601,0.027615,0.007586,0.000457,-0.004401,-0.048330,0.020491,0.005789,-0.016217,-0.002392
volatile acidity,-0.003464,0.003830,0.016014,0.007062,0.007724,-0.001479,0.002375,-0.005397,-0.015536,0.004679,0.003219,-0.009824,0.002820
citric acid,0.001064,-0.006601,0.007062,0.016673,0.001516,-0.003296,0.001970,-0.003154,-0.003377,0.002088,-0.000995,-0.003029,-0.000656
residual sugar,0.001149,0.027615,0.007724,0.001516,0.060160,0.007048,0.002045,-0.017084,-0.062019,0.020598,0.010205,-0.024401,-0.004937
chlorides,-0.004029,0.007586,-0.001479,-0.003296,0.007048,0.011606,-0.001255,-0.001296,-0.012881,0.005123,0.001178,-0.001956,-0.000166
free sulfur dioxide,0.007154,0.000457,0.002375,0.001970,0.002045,-0.001255,0.032979,-0.015920,-0.004918,0.000009,-0.000551,0.000490,-0.005732
total sulfur dioxide,0.003731,-0.004401,-0.005397,-0.003154,-0.017084,-0.001296,-0.015920,0.032356,0.018193,-0.003824,-0.005664,0.012171,0.002650
density,0.011294,-0.048330,-0.015536,-0.003377,-0.062019,-0.012881,-0.004918,0.018193,0.113798,-0.033484,-0.017730,0.046788,0.007419
pH,-0.006263,0.020491,0.004679,0.002088,0.020598,0.005123,0.000009,-0.003824,-0.033484,0.021890,0.002495,-0.013747,-0.002304


In [29]:
# Standard Error
se = np.sqrt(C.diagonal())

In [30]:
# Z critical value
z_stats_hand = []
p_values_hand = []
z_dist = stats.norm(0, 1).pdf
for i in range(len(se)):
    z_stat = log_reg.weights[i] / se[i]
    z_stats_hand.append(z_stat)
    if z_stat <= 0:
        p_val = integrate.quad(z_dist, -np.inf, z_stat)
    else:
        p_val = integrate.quad(z_dist, z_stat, np.inf)
    p_values_hand.append(p_val[0])

In [31]:
# P value
print('Hand-Coded Model Statistics')
final_stats_df = pd.DataFrame()
final_stats_df['Hand-Coded Model Standard Error'] = se
final_stats_df['Hand-Coded Model Z-Statistics'] = z_stats_hand
final_stats_df['Hand-Coded Model P-Values'] = p_values_hand
final_stats_df.index = x_train.columns
final_stats_df

Hand-Coded Model Statistics


,Hand-Coded Model Standard Error,Hand-Coded Model Z-Statistics,Hand-Coded Model P-Values
const,0.144938,20.364898,1.712828e-92
fixed acidity,0.203441,-3.978219,3.471667e-05
volatile acidity,0.126545,-8.167536,1.573754e-16
citric acid,0.129124,3.102437,9.596709e-04
residual sugar,0.245276,3.977539,3.481606e-05
chlorides,0.107732,-7.294078,1.503551e-13
free sulfur dioxide,0.181600,0.896082,1.851044e-01
total sulfur dioxide,0.179877,11.200214,2.033752e-29
density,0.337339,-3.682276,1.155805e-04
pH,0.147952,-4.152735,1.642624e-05


In [32]:
print(f'Library Model Statistics')
final_stats_df = pd.DataFrame()
final_stats_df['Library Model Standard Error'] = model.bse
final_stats_df['Library Model Z-Statistics'] = model.tvalues
final_stats_df['Library Model P-Values'] = model.pvalues
final_stats_df.index = x_train.columns
final_stats_df

Library Model Statistics


,Library Model Standard Error,Library Model Z-Statistics,Library Model P-Values
const,0.390072,12.825131,1.185856e-37
fixed acidity,0.344401,0.282117,7.778540e-01
volatile acidity,0.211644,-5.156848,2.511413e-07
citric acid,0.205225,2.907530,3.642958e-03
residual sugar,0.648480,8.477357,2.303664e-17
chlorides,0.185798,-4.816389,1.461796e-06
free sulfur dioxide,0.314078,-3.575605,3.494184e-04
total sulfur dioxide,0.344706,8.989922,2.474062e-19
density,0.640851,-8.322859,8.586685e-17
pH,0.265783,0.536237,5.917950e-01
